In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from subprocess import check_output


In [2]:
data = pd.read_csv('https://aaa-pgdmlai.s3.amazonaws.com/member_refined.csv')
data.columns

Index(['Household_Key', 'ZIP5', 'FSV_CMSI_Flag', 'FSV_Credit_Card_Flag',
       'FSV_Deposit_Program_Flag', 'FSV_Home_Equity_Flag', 'FSV_ID_Theft_Flag',
       'FSV_Mortgage_Flag', 'INS_Client_Flag', 'TRV_Globalware_Flag',
       'Length_Of_Residence', 'Income', 'Children',
       'Do_Not_Direct_Mail_Solicit', 'Email_Available', 'ERS_ENT_Count_Year_1',
       'ERS_ENT_Count_Year_2', 'ERS_ENT_Count_Year_3',
       'ERS_Member_Cost_Year_1', 'ERS_Member_Cost_Year_2',
       'ERS_Member_Cost_Year_3', 'Member_Tenure_Years', 'Opt_Out_Publication',
       'Mosaic_Household', 'Mosaic_Global_Household',
       'kcl_B_IND_MosaicsGrouping', 'New_Mover_Flag', 'Breakdown_City',
       'Basic_Cost', 'Calculated_Tow_Miles', 'Call_Canceled', 'Call_Killed',
       'Cash_Call', 'Clearing_Code_Last_Description', 'Fleet_Indicator',
       'Is_Duplicate', 'Is_NSR', 'Member_Match_Flag', 'Motorcycle_Indicator',
       'Plus_Cost', 'Premier_Cost', 'Prob1_Code_Description',
       'SC_Vehicle_Manufacturer_Name

In [3]:
data.shape

(21344, 47)

In [4]:
data.describe()

,Household_Key,ZIP5,Length_Of_Residence,Do_Not_Direct_Mail_Solicit,Email_Available,ERS_ENT_Count_Year_1,ERS_ENT_Count_Year_2,ERS_ENT_Count_Year_3,ERS_Member_Cost_Year_1,ERS_Member_Cost_Year_2,...,Basic_Cost,Calculated_Tow_Miles,Is_Duplicate,Is_NSR,Member_Match_Flag,Plus_Cost,Premier_Cost,Total_Cost,Was_Duplicated,Was_Towed_To_AAR_Referral
count,2.134400e+04,21344.000000,12010.000000,21343.000000,21343.000000,16675.000000,16675.000000,16675.000000,16675.000000,16675.000000,...,13944.000000,13861.000000,13997.0,13997.000000,13997.0,13944.000000,13911.000000,13944.000000,13997.000000,13997.000000
mean,1.597040e+07,2944.899831,11.524979,0.052851,0.536101,0.518561,0.936252,0.959760,28.068167,50.970628,...,42.629593,2.379071,0.0,0.059156,1.0,3.450947,0.006901,47.273696,0.065157,0.013646
std,2.138079e+07,478.757704,5.495280,0.223741,0.498707,0.985532,1.253213,1.322932,57.576978,78.213543,...,15.713987,7.030493,0.0,0.235924,0.0,18.213483,0.692848,86.869110,0.246811,0.116020
min,8.750000e+02,2801.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.350387e+06,2857.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,30.000000,0.000000,0.0,0.000000,1.0,0.000000,0.000000,30.500000,0.000000,0.000000
50%,9.811235e+06,2888.000000,15.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,53.000000,0.000000,0.0,0.000000,1.0,0.000000,0.000000,53.000000,0.000000,0.000000
75%,1.654012e+07,2910.000000,15.000000,0.000000,1.000000,1.000000,2.000000,1.000000,36.000000,66.750000,...,53.000000,1.000000,0.0,0.000000,1.0,0.000000,0.000000,53.000000,0.000000,0.000000
max,1.000791e+08,6902.000000,49.000000,1.000000,1.000000,7.000000,10.000000,9.000000,560.740000,904.390000,...,106.390000,142.000000,0.0,1.000000,1.0,530.000000,81.000000,9869.000000,1.000000,1.000000


In [5]:
new_data = data.copy()
cols_with_missing = (col for col in new_data.columns 
                                 if new_data[col].isnull().any())
for col in cols_with_missing:
    new_data[col + '_was_missing'] = new_data[col].isnull()
data_encoded = pd.get_dummies(new_data)
my_imputer = SimpleImputer()
new_data = pd.DataFrame(my_imputer.fit_transform(data_encoded))
new_data.columns = data_encoded.columns

In [6]:
new_data.iloc[:,5:].head(5)

,ERS_ENT_Count_Year_1,ERS_ENT_Count_Year_2,ERS_ENT_Count_Year_3,ERS_Member_Cost_Year_1,ERS_Member_Cost_Year_2,ERS_Member_Cost_Year_3,Member_Tenure_Years,Basic_Cost,Calculated_Tow_Miles,Is_Duplicate,...,SC_Vehicle_Model_Name_YARIS,SC_Vehicle_Model_Name_YUKON,SC_Vehicle_Model_Name_YUKON XL,SC_Vehicle_Model_Name_YUKON XL 1500,SC_Vehicle_Model_Name_YUKON XL 2500,SC_Vehicle_Model_Name_Yukon XL 1500,SC_Vehicle_Model_Name_Z3,SC_Vehicle_Model_Name_Z4,SC_Vehicle_Model_Name_ZEPHYR,SC_Vehicle_Model_Name_murano
0,0.518561,0.936252,0.95976,28.068167,50.970628,49.518118,32.956239,42.629593,2.379071,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,2.00000,0.000000,0.000000,65.000000,49.000000,32.500000,0.800000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,2.00000,0.000000,0.000000,65.000000,49.000000,30.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,2.00000,0.000000,0.000000,65.000000,49.000000,32.500000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,2.00000,0.000000,0.000000,65.000000,49.000000,30.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
labels = np.array(new_data['Total_Cost'])

In [8]:
new_data= new_data.drop('Total_Cost', axis = 1)

In [9]:
feature_list = list(new_data.columns)

In [10]:
features = np.array(new_data)

In [11]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [12]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (16008, 1499)
Training Labels Shape: (16008,)
Testing Features Shape: (5336, 1499)
Testing Labels Shape: (5336,)


In [13]:
baseline_preds = test_features[:, feature_list.index('Premier_Cost')]

In [14]:
baseline_errors = abs(baseline_preds - test_labels)

In [15]:
print('Average baseline error: ', round(np.mean(baseline_errors), 2))

Average baseline error:  48.65


In [16]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

In [17]:
rf.fit(train_features, train_labels);

In [18]:
predictions = rf.predict(test_features)

In [19]:
errors = abs(predictions - test_labels)

In [23]:
print('Mean Absolute Error:', round(np.mean(errors), 2))

Mean Absolute Error: 2.36


In [35]:
mape = 100 * (errors/test_labels)

4.073239387114041e-12


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [39]:
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: nan %.
